<a href="https://colab.research.google.com/github/DeshanHarshana/Sinhala-News-Classification/blob/main/Preprocessing_sinhala_text_contents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/datasets/

Mounted at /content/drive
/content/drive/My Drive/datasets


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [119]:
df = pd.read_csv('Contents.csv')
data=df.dropna()
data.columns =['News', 'Label'] # add labels

In [120]:
with open('stopwords.txt', 'r') as f:
    sinhala_stopwords = [line.strip() for line in f]
with open('sufix_list.txt', 'r') as f:
    suffix_list = [line.strip() for line in f]

In [121]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
#cleaning
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(sinhala_stopwords)
SUFFIX = set(suffix_list)

def clean_text(text):
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    #text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('\\', '')
    text = text.replace('-', '')
    text = text.replace('\'', '')
    text = text.replace('.', ' ')
    text = text.replace('“', '')
    text = text.replace('”', '')
    
    text = ''.join([i for i in text if not i.isdigit()])
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = ' '.join(word for word in text.split() if word not in SUFFIX)
    return text

data['News'] = data['News'].progress_apply(clean_text)

  0%|          | 0/28911 [00:00<?, ?it/s]

<ipython-input-121-3c2415c49491>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [112]:
def reEncode(text):
  tokens = nltk.word_tokenize(text)
  regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
  tokens = [regex.sub('', w) for w in tokens]
  return ' '.join(tokens)


In [91]:
#data['News'] = data.progress_apply(lambda x: reEncode(x['News']), axis=1)

  0%|          | 0/3 [00:00<?, ?it/s]

In [122]:
with open('stem_dictonary.txt') as f:
    lines = [line.strip() for line in f]
stem_list=[]
for i in lines:
  x = i.split("\t")
  stem_list.append(x)

In [101]:
#for i in tqdm(range(0,len(stem_list))):
#  try:
#    stem_list[i][0]=reEncode(stem_list[i][0])
#    stem_list[i][1]=reEncode(stem_list[i][1])
#  except:
#    pass

100%|██████████| 27201/27201 [00:03<00:00, 8516.85it/s]


In [123]:
def stemming(string1):
  wordlist=word_tokenize(string1)
  for i in range(len(wordlist)):
    for j in stem_list:
      if j[0]==wordlist[i]:
        wordlist[i]=j[1]

  return ' '.join(wordlist)

In [115]:
string1="ක්‍රීඩකයන්"
print(stemming(string1))

ක්‍රීඩකයා


In [124]:
data['News'] = data.progress_apply(lambda x: stemming(x['News']), axis=1)
#


  0%|          | 0/28911 [00:00<?, ?it/s]

<ipython-input-124-640f25d17b8a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
data.head()

,News,Label
0,එංගලන්තයේ හන්ඩ්‍රඩ් තරඟාවලියේ සදර්න් බ්‍රේව් ක...,1
1,එංගලන්ත පාකිස්ථානය රාවල්පිණ්ඩි පැවැත්වෙන පළමු ...,1
2,සංචාරක ඇෆ්ගනිස්ථාන කණ්ඩාය ලංකා පැවැත්වෙන තරග ත...,1
3,පසුගිය දා ඕස්ට්‍රේලියාවේ පැවති පන්දුවාර ලෝක කු...,1
4,අන්තර් සමාජ තුන් දින ලීග ක්‍රිකට් තරගාවලියේ අව...,1


In [129]:
data.to_csv('preprocessed_content.csv', index=False)

In [136]:
%ls

 beautfulsoup.py    DataScience/                  preprocessed_content.csv
 cache_dir/        'fasttext_trained model.bin'   result/
 cc.si.300.bin      file_name.csv                 runs/
 cc.si.300.vec      file.txt                      stem_dictonary.txt
 cleaned_data.csv   news.test                     stopwords.txt
 Contents.csv       news.train                    sufix_list.txt
 data.csv           outputs/


In [137]:
df = pd.read_csv('preprocessed_content.csv')

In [139]:
df.tail()


,News,Label
28906,ලංකා පාකිස්ථානය ගාල්ල ක්‍රිකට් ක්‍රීඩාංගනයේ පැ...,3
28907,කොළඹ කොටස් වෙළඳපොල ඊයේ ඉහළ යාමක පෙන්නු කළ දිනය...,3
28908,එංගලන්තයේ වාර්තාගත උෂ්ණත්වයක් ඊයේ සනිටුහන් වී ...,3
28909,සෞඛ්‍ය සේවකයන්ට නිසි ඉන්ධන ලබාදීමේ ක්‍රමවේදයක්...,3
28910,ජාතික ඉන්ධන අවසර පත්‍රය ජූලි සඳුදා ක්‍රියාත්මක...,3
